# Label modifcation by Kamiran et al. - Recruiting data

This notebook contains the implementation of the post-processing label modification algorithm introduced in [Decision Theory for Discrimination-aware Classification](https://ieeexplore.ieee.org/abstract/document/6413831) by Kamiran et al. (2012) as part of the IBM AIF360 fairness tool box github.com/IBM/AIF360.

The intervention method identifies a low-confidence subset of the data for probabilistic classifiers, and produces new predicted labels for this subset by assigning positive labels to the unprivileged group and negative labels to the privileged group. Different notions of fairness can be achieved in this way, including demographic parity, equalised odds and equal opportunity, of which each one is allowed in the implementation. In this notebook we demonstrate each of these three definitions as a result of the fairness intervention. 

For simplicity, we'll focus mitigating bias with resepct to sex.

In [ ]:
from pathlib import Path

import joblib
import pandas as pd
from aif360.algorithms.postprocessing.reject_option_classification import (
    RejectOptionClassification,
)
from aif360.datasets import StandardDataset
from fairlearn.metrics import (
    demographic_parity_difference,
    equalized_odds_difference,
)
from helpers.metrics import accuracy
from helpers.plot import group_bar_plots

In [ ]:
from helpers import export_plot

## Load data

We have committed preprocessed data to the repository for reproducibility and we load it here. Check out the preprocessing notebook for details on how this data was obtained.

In [ ]:
artifacts_dir = Path("../../../artifacts")

In [ ]:
# override data_dir in source notebook
# this is stripped out for the hosted notebooks
artifacts_dir = Path("../../../../artifacts")

In [ ]:
data_dir = artifacts_dir / "data" / "recruiting"

train = pd.read_csv(data_dir / "processed" / "train.csv")
val = pd.read_csv(data_dir / "processed" / "val.csv")
test = pd.read_csv(data_dir / "processed" / "test.csv")

In order to process data for our fairness intervention we need to define special dataset objects which are part of every intervention pipeline within the IBM AIF360 toolbox. These objects contain the original data as well as some useful further information, e.g., which feature is the protected attribute as well as which column corresponds to the label.

In [ ]:
train_sds = StandardDataset(
    train,
    label_name="employed_yes",
    favorable_classes=[1],
    protected_attribute_names=["race_white"],
    privileged_classes=[[1]],
)
test_sds = StandardDataset(
    test,
    label_name="employed_yes",
    favorable_classes=[1],
    protected_attribute_names=["race_white"],
    privileged_classes=[[1]],
)
val_sds = StandardDataset(
    val,
    label_name="employed_yes",
    favorable_classes=[1],
    protected_attribute_names=["race_white"],
    privileged_classes=[[1]],
)
index = train_sds.feature_names.index("race_white")

Define which binary value goes with the (un-)privileged group

In [ ]:
privileged_groups = [{"race_white": 1.0}]
unprivileged_groups = [{"race_white": 0.0}]

## Train unfair model

For maximum reproducibility we load the baseline model from disk, but the code used to train can be found in the baseline model notebook.

In [ ]:
baseline_model = joblib.load(
    artifacts_dir / "models" / "recruiting" / "baseline.pkl"
)

Get predictions for the validation and test data

In [ ]:
bl_val_probs = baseline_model.predict_proba(val.drop("employed_yes", axis=1))[
    :, 1
]
val_sds_pred = val_sds.copy(deepcopy=True)
val_sds_pred.scores = bl_val_probs.reshape(-1, 1)

bl_test_probs = baseline_model.predict_proba(
    test.drop("employed_yes", axis=1)
)[:, 1]
bl_test_pred = bl_test_probs > 0.5
test_sds_pred = test_sds.copy(deepcopy=True)
test_sds_pred.scores = bl_test_probs.reshape(-1, 1)

## Demographic parity

We first address demographic parity. In order to do so, we learn the label modification algorithm based on the true and predicted labels of the validation data. We then apply the learnt intervention to the predictions of the test data and analyse the outcomes for fairness and accuracy.

In [ ]:
# Metric used
metric_name = "Statistical parity difference"  # Alias for demographic parity

# Upper and lower bound on the fairness metric used
metric_ub = 0.05
metric_lb = -0.05

Learn intervention on validation data.

In [ ]:
ROC = RejectOptionClassification(
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups,
    low_class_thresh=0.01,
    high_class_thresh=0.99,
    num_class_thresh=100,
    num_ROC_margin=50,
    metric_name=metric_name,
    metric_ub=metric_ub,
    metric_lb=metric_lb,
)
ROC = ROC.fit(val_sds, val_sds_pred)

Apply intervention on test data.

In [ ]:
# Transform the test set
test_sds_pred_transf = ROC.predict(test_sds_pred).copy(deepcopy=True)
test_pred_labels = test_sds_pred_transf.labels.flatten()

Analyse fairness and accuracy on test data.

In [ ]:
test_features = test.drop(columns=["race_white", "employed_yes"]).values
test_race = test.race_white.values
test_employed = test.employed_yes.values
mask = test_race == 1

# baseline metrics
bl_test_acc = accuracy(test_employed, bl_test_probs)
bl_test_dpd = demographic_parity_difference(
    test.employed_yes, bl_test_pred, sensitive_features=test_race,
)

# new model metrics
test_acc = accuracy(test_employed, test_pred_labels)
test_dpd = demographic_parity_difference(
    test.employed_yes, test_pred_labels, sensitive_features=test_race,
)

print(f"Baseline accuracy: {bl_test_acc:.3f}")
print(f"Accuracy: {test_acc:.3f}\n")

print(f"Baseline demographic parity: {bl_test_dpd:.3f}")
print(f"Demographic parity: {test_dpd:.3f}\n")

Consider accuracy on the white / black subgroups

In [ ]:
white_acc = accuracy(
    test_pred_labels[test.race_white == 1],
    test.employed_yes[test.race_white == 1],
)
black_acc = accuracy(
    test_pred_labels[test.race_white == 0],
    test.employed_yes[test.race_white == 0],
)

mean_black_score = test_pred_labels[test.race_white == 0].mean()
mean_white_score = test_pred_labels[test.race_white == 1].mean()

print(f"Black accuracy: {black_acc:.3f}")
print(f"White accuracy: {white_acc:.3f}")
print(f"Mean black score: {mean_black_score:.3f}")
print(f"Mean white score: {mean_white_score:.3f}")

In [ ]:
dp_bar = group_bar_plots(
    test_pred_labels,
    test.race_white.map({0: "Black", 1: "White"}),
    title="Predictions by race",
    xlabel="Proportion predicted successful",
)
dp_bar

In [ ]:
export_plot(dp_bar, "kamiran-dp.json")

## Equalised Odds

We'll now repeat the process for equalised odds, which requires us changing the underlying metric which leads to learning the correction according a new decision threshold addressing equalised odds. There are no further modifcations to the existing parameter choices required. However we apply smaller bounds on equalised odds than for demographic parity.

In [ ]:
# Metric used
metric_name = "Average odds difference"  # alias for equalised odds

# Upper and lower bound on the fairness metric used
metric_ub = 0.01
metric_lb = -0.01

Learn intervention on validation data.

In [ ]:
ROC = RejectOptionClassification(
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups,
    low_class_thresh=0.01,
    high_class_thresh=0.99,
    num_class_thresh=100,
    num_ROC_margin=50,
    metric_name=metric_name,
    metric_ub=metric_ub,
    metric_lb=metric_lb,
)
ROC = ROC.fit(val_sds, val_sds_pred)

Apply intervention on test data.

In [ ]:
# Transform the test set
test_sds_pred_transf = ROC.predict(test_sds_pred).copy(deepcopy=True)
test_pred_labels = test_sds_pred_transf.labels.flatten()

Analyse fairness and accuracy on test data.

In [ ]:
test_features = test.drop(columns=["race_white", "employed_yes"]).values
test_race = test.race_white.values
test_employed = test.employed_yes.values
mask = test_race == 1

# baseline metrics
bl_test_acc = accuracy(test_employed, bl_test_probs)
bl_test_eod = equalized_odds_difference(
    test_employed, bl_test_pred, sensitive_features=test_race,
)

# new model metrics
test_acc = accuracy(test_employed, test_pred_labels)
test_eod = equalized_odds_difference(
    test_employed, test_pred_labels, sensitive_features=test_race,
)

print(f"Baseline accuracy: {bl_test_acc:.3f}")
print(f"Accuracy: {test_acc:.3f}\n")

print(f"Baseline equalised odds (dist.): {bl_test_eod:.3f}")
print(f"Equalised odds (dist.): {test_eod:.3f}\n")

In [ ]:
eo_bar = group_bar_plots(
    test_pred_labels,
    test.race_white.map({0: "Black", 1: "White"}),
    groups=test.employed_yes,
    group_names=["Not employed", "Employed"],
    title="Predictions by race and outcome",
    xlabel="Proportion predicted successful",
    ylabel="Outcome",
)
eo_bar

In [ ]:
export_plot(eo_bar, "kamiran-eo.json")

## Equal opportunity

We'll now repeat the process for equal opportunity, which only requires us changing the underlying metric.

In [ ]:
# Metric used
metric_name = "Equal opportunity difference"

# Upper and lower bound on the fairness metric used
metric_ub = 0.01
metric_lb = -0.01

Learn interrvention on validation data.

In [ ]:
ROC = RejectOptionClassification(
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups,
    low_class_thresh=0.01,
    high_class_thresh=0.99,
    num_class_thresh=100,
    num_ROC_margin=50,
    metric_name=metric_name,
    metric_ub=metric_ub,
    metric_lb=metric_lb,
)
ROC = ROC.fit(val_sds, val_sds_pred)

Apply intervention on test data.

In [ ]:
# Transform the test set
test_sds_pred_transf = ROC.predict(test_sds_pred).copy(deepcopy=True)
test_pred_labels = test_sds_pred_transf.labels.flatten()

Analyse fairness and accuracy on test data.

In [ ]:
test_features = test.drop(columns=["race_white", "employed_yes"]).values
test_race = test.race_white.values
test_employed = test.employed_yes.values
mask = test_race == 1

# baseline metrics
bl_test_acc = accuracy(test_employed, bl_test_probs)
bl_test_eoppd = equalized_odds_difference(
    test_employed[test.employed_yes == 1],
    bl_test_pred[test.employed_yes == 1],
    sensitive_features=test_race[test.employed_yes == 1],
)

# new model metrics
test_acc = accuracy(test_employed, test_pred_labels)
test_eoppd = equalized_odds_difference(
    test_employed[test.employed_yes == 1],
    test_pred_labels[test.employed_yes == 1],
    sensitive_features=test_race[test.employed_yes == 1],
)

print(f"Baseline accuracy: {bl_test_acc:.3f}")
print(f"Accuracy: {test_acc:.3f}\n")

print(f"Baseline equal opportunity: {bl_test_eoppd:.3f}")
print(f"Equal opportunity: {test_eoppd:.3f}\n")

In [ ]:
eopp_bar = group_bar_plots(
    test_pred_labels[test.employed_yes == 1],
    test.race_white[test.employed_yes == 1].map({0: "Black", 1: "White"}),
    title="Predictions by race for successful applicants",
    xlabel="Proportion predicted successful",
)
eopp_bar

In [ ]:
export_plot(eopp_bar, "kamiran-eopp.json")